任务：图像分类网络的常用的网络层、如何处理图片数据以及整体程序的结构
图片分类任务步骤：
获取数据：使用内部API下载数据集再进行解析，存入ImageDataGenerator迭代器中
数据预处理：归一化
构建神经网络模型：图片常用到Conv2D卷积层和MaxPolling2D池化层
编译模型、训练模型
可视化结果训练过程，训练中的参数保存在history中，只要用工具画这个即可

数据集： Kaggle 上的猫狗分类数据集
- 训练集合 2000 张图片，包括 1000 张狗图片和 1000 张猫图片；
- 验证（测试）集合 1000 张图片，包括 500 张狗图片和 500 张猫图片。

注意：我的PC在运行代码块的时候，带不起来，然后就崩了.但是放在云平台上能运行，结果在最后，
这里输出的图片线上没有标签，和预想的不一样，后面可以把matplotlib画线的库给学习一下，


In [1]:
import tensorflow as tf
import os  #os 用于目录的相关操作
import matplotlib.pyplot as plt  #plt 用于我们的图片绘制工作

# 定义一些数据集合与训练的基本参数，留作后面训练之用
BATCH_SIZE = 64 #批次的大小，我们会将 BATCH_SIZE 个图片数据同时送给模型进行训练，这个数值大家可以根据自己的内存大小或者显存大小进行调整
TRAIN_NUM = 2000 #训练集的数量
VALID_NUM = 1000 #验证集的数量 
EPOCHS = 15 # 训练的周期数，将所有的数据通用模型训练一遍称为一个 EPOCH，一般来说，EPOCH 的数值越大，模型在训练集合上的准确率越高，相应的所需要的时间也更长
Height = 128 # 图片的高度
Width = 128 # 图片的长度


# 进行数据的下载与解压
dataset_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
# 使用 Keras 内置的下载工具进行下载，返回的是数据集所在的路径
path_download = os.path.dirname(tf.keras.utils.get_file('cats_and_dogs.zip',
                                origin=dataset_url,
                                extract=True))

# 结合数据集的下载位置与其内部文件建构得到训练数据集与测试数据集所在的目录
train_dataset_dir = path_download + '/cats_and_dogs_filtered/train'
valid_dataset_dir = path_download + '/cats_and_dogs_filtered/validation'

# 可以通过下面的方法来查看数据文件所在的目录
# 报错：name 'cat_train_dir' is not defined
# print(cat_train_dir, cat_validation_dir, dog_train_dir, dog_validation_dir)


# 这里我们要使用 Keras 内部内置的 ImageDataGenerator 来作为迭代器产生数据
'''
在这里我们定义了两个图片数据迭代器，同时我们定义了 rescale 参数，
在函数内部每张图片的每个像素数据都会乘以 1./255，以将其归一化到 [0, 1] 之间，
因为机器学习之中模型的最好输入数据是在 [0, 1] 之间。
'''
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


   在这里我们定义了两个图片数据迭代器，同时我们定义了 rescale 参数，在函数内部每张图片的每个像素数据都会乘以 1./255，以将其归一化到 [0, 1] 之间，因为机器学习之中模型的最好输入数据是在 [0, 1] 之间。

然后我们使用文件夹中的数据来初始化这两个图片数据迭代器，这个功能是采用 flow_from_directory 函数来实现的：

In [4]:
'''
使用 Keras 内部内置的 ImageDataGenerator 来作为迭代器产生数据
batch_size：数据批次的大小，我们之前定义为 64；
directory：数据存放的文件夹；
shuffle：数据是否打乱，这里我们选择进行打乱；
target_size：输出图片的大小，这里我们将长宽都定义为 128；
class_mode：分类模式，这里我们选择 “binary” 表示这是一个二分类问题，
            如果是多分类问题，我们可以选择 “categorical”。
'''
train_data_generator=train_image_generator.flow_from_directory(
                                batch_size=BATCH_SIZE,
                                directory=train_dataset_dir,
                                shuffle=True,
                                target_size=(Height,Width),
                                class_mode="binary"
                )

valid_dataset_generator=valid_image_generator.flow_from_directory(
                                batch_size=BATCH_SIZE,
                                directory=valid_dataset_dir,
                                shuffle=True,
                                target_size(Height,Width)
                                class_mode="binary"
                                )

SyntaxError: invalid syntax (2499953817.py, line 23)

In [ ]:
'''
构建模型,这里我们采用传统的顺序结构来定义我们的网络结构
Conv2D：卷积层，用来提图片的特征；
MaxPooling2D：池化层，用来降低计算量；
一般来说，池化层都会跟在卷积层之后。
'''
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,3,padding="same",activation="relu",
                           input_shape=(Height,Width,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32,3,padding="same",activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64,3,padding="same",activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation="relu"),
    tf.keras.layers.Dense(1)
])

In [5]:
'''
然后我们对模型进行编译：
优化器我们选择最常用的 adam 优化器，同时我们在训练的过程中记录准确率（accuracy）；
因为任务是二元分类，因此我们采用的是二元交叉熵（BinaryCrossentropy）；
因为网络最后一层没有激活函数而直接输出，因此模型产生的数据实际是两个类别的可能性的大小；
所以优化器会加上参数 from_logits=True，以表示输出为可能性大小，而不是具体类别
'''
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
model.summary()

'\n\n'

运行上面代码我们可以得到网络的结构：
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   

conv2d_9 (Conv2D)            (None, 128, 128, 16)      448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               8389120   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 513       

Total params: 8,413,217
Trainable params: 8,413,217
Non-trainable params: 0




In [6]:
'''
在训练模型的时候，我们会使用之前定义好的图片数据迭代器，
同时将训练数据保存在 history 对象之中：
'''
history=model.fit_generator(
    train_image_generator,
    steps_per_epoch=TRAIN_NUM // BATCH_SIZE,
    epochs=EPOCHS,
    valid_dataset_data=valid_image_generator,
    valid_steps=VALID_NUM // BATCH_SIZE
)


NameError: name 'model' is not defined

通过训练，我们可以得到以下的输出：
Epoch 1/15
31/31 [==============================] - 41s 1s/step - loss: 0.7072 - accuracy: 0.5134 - val_loss: 0.6650 - val_accuracy: 0.5167
Epoch 2/15
31/31 [==============================] - 40s 1s/step - loss: 0.6540 - accuracy: 0.5826 - val_loss: 0.6381 - val_accuracy: 0.5448
Epoch 3/15
31/31 [==============================] - 39s 1s/step - loss: 0.5780 - accuracy: 0.6844 - val_loss: 0.5859 - val_accuracy: 0.7208
Epoch 4/15
31/31 [==============================] - 40s 1s/step - loss: 0.5245 - accuracy: 0.7485 - val_loss: 0.5550 - val_accuracy: 0.6719
Epoch 5/15
31/31 [==============================] - 40s 1s/step - loss: 0.4673 - accuracy: 0.7645 - val_loss: 0.5654 - val_accuracy: 0.6865
Epoch 6/15
31/31 [==============================] - 40s 1s/step - loss: 0.3968 - accuracy: 0.8110 - val_loss: 0.5929 - val_accuracy: 0.7208
Epoch 7/15
31/31 [==============================] - 40s 1s/step - loss: 0.3216 - accuracy: 0.8492 - val_loss: 0.6224 - val_accuracy: 0.7104
Epoch 8/15
31/31 [==============================] - 40s 1s/step - loss: 0.2577 - accuracy: 0.8879 - val_loss: 0.6871 - val_accuracy: 0.7115
Epoch 9/15
31/31 [==============================] - 40s 1s/step - loss: 0.2204 - accuracy: 0.9060 - val_loss: 0.6982 - val_accuracy: 0.7250
Epoch 10/15
31/31 [==============================] - 40s 1s/step - loss: 0.1633 - accuracy: 0.9329 - val_loss: 0.9962 - val_accuracy: 0.6896
Epoch 11/15
31/31 [==============================] - 40s 1s/step - loss: 0.1371 - accuracy: 0.9489 - val_loss: 0.8724 - val_accuracy: 0.6990
Epoch 12/15
31/31 [==============================] - 40s 1s/step - loss: 0.0937 - accuracy: 0.9654 - val_loss: 1.1101 - val_accuracy: 0.7052
Epoch 13/15
31/31 [==============================] - 40s 1s/step - loss: 0.0640 - accuracy: 0.9742 - val_loss: 1.0343 - val_accuracy: 0.7083
Epoch 14/15
31/31 [==============================] - 40s 1s/step - loss: 0.0449 - accuracy: 0.9866 - val_loss: 1.1627 - val_accuracy: 0.7167
Epoch 15/15
31/31 [==============================] - 40s 1s/step - loss: 0.0199 - accuracy: 0.9954 - val_loss: 1.2627 - val_accuracy: 0.7156

输出准确率与损失值因人而异，在这里我们在训练集合上得到了 99.54% 的准确率，在验证集上得到了 71.56% 的准确率.


In [ ]:
'''
可视化训练过程
在上一步之中，我们特地将训练过程的数据记录进了 history 对象之中；
history 对象中的 history 数据对象是一个字典型的结构，
其中包含了我们在训练过程中的准确率与损失值等等
于是我们将其可视化：plot 折线图



最后图显示的结果分析：
由此可以看出，随着训练的不断迭代，训练集合上的准确率不断上升，损失值不断下降；
但是验证集上的准确率在第 3 个 Epoch 以后便趋于平稳，
而损失值却在第 3 个 Epoch 之后逐渐上升。
这就是我们在训练过程中遇到的过拟合，我们以后会有课程详细介绍过拟合
'''
acc=history.history["accuracy"]
loss=history.history['loss']

val_acc=history.history['val_accuracy']
val_loss=history.history['val_loss']

# 第一个图片展示准确率的变化
plt.plot(range(EPOCHS),acc,label="Train ACC")
plt.plot(range(EPOCHS),val_acc,label="Valid ACC")
plt.show()

# 第二个图片展示损失值的变化
plt.plot(range(EPOCHS),loss,label='Train loss')
plt.plot(range(EPOCHS),val_loss,label='Valid Loss')
plt.show()



In [ ]:
'''
完整可运行代码：

import tensorflow as tf
import os
import matplotlib.pyplot as plt

dataset_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_download = os.path.dirname(tf.keras.utils.get_file('cats_and_dogs.zip', origin=dataset_url, extract=True))
train_dataset_dir = path_download + '/cats_and_dogs_filtered/train'
valid_dataset_dir = path_download + '/cats_and_dogs_filtered/validation'
cat_train_dir = path_download + '/cats_and_dogs_filtered/train/cats'
cat_validation_dir = path_download + '/cats_and_dogs_filtered/validation/cats'
dog_train_dir = path_download + '/cats_and_dogs_filtered/train/dogss'
dog_validation_dir = path_download + '/cats_and_dogs_filtered/validation/dogs'

BATCH_SIZE = 64
TRAIN_NUM = 2000
VALID_NUM = 1000
EPOCHS = 15
Height = 128
Width = 128

train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data_generator = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                              directory=train_dataset_dir,
                              shuffle=True,
                              target_size=(Height, Width),
                              class_mode='binary')
valid_data_generator = valid_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                              directory=valid_dataset_dir,
                              shuffle=True,
                              target_size=(Height, Width),
                              class_mode='binary')

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu',
                input_shape=(Height, Width ,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
       loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
       metrics=['accuracy'])
model.summary()

history = model.fit_generator(
    train_data_generator,
    steps_per_epoch=TRAIN_NUM // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=valid_data_generator,
    validation_steps=VALID_NUM // BATCH_SIZE)

acc = history.history['accuracy']
loss=history.history['loss']

val_acc = history.history['val_accuracy']
val_loss=history.history['val_loss']

epochs_ran = range(EPOCHS)

plt.plot(epochs_ran, acc, label='Train Acc')
plt.plot(epochs_ran, val_acc, label='Valid Acc')
plt.show()

plt.plot(epochs_ran, loss, label='Train Loss')
plt.plot(epochs_ran, val_loss, label='Valid Loss')
plt.show()

'''

在云平台上运行的结果：
Downloading data from https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
68608000/68606236 [==============================] - 8s 0us/step
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.

使用model.summary()打印出来模型的构建情况：
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               8389120   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 8,413,217
Trainable params: 8,413,217
Non-trainable params: 0

模型训练的过程，模型没有评估：
Epoch 1/15
31/31 [==============================] - 563s 18s/step - loss: 0.8026 - accuracy: 0.5015 - val_loss: 0.6858 - val_accuracy: 0.5042
Epoch 2/15
31/31 [==============================] - 356s 11s/step - loss: 0.6750 - accuracy: 0.5325 - val_loss: 0.6383 - val_accuracy: 0.6104
Epoch 3/15
31/31 [==============================] - 352s 11s/step - loss: 0.6306 - accuracy: 0.6121 - val_loss: 0.6086 - val_accuracy: 0.6583
Epoch 4/15
31/31 [==============================] - 328s 11s/step - loss: 0.5579 - accuracy: 0.6829 - val_loss: 0.6179 - val_accuracy: 0.6740
Epoch 5/15
31/31 [==============================] - 335s 11s/step - loss: 0.4930 - accuracy: 0.7438 - val_loss: 0.5693 - val_accuracy: 0.7354
Epoch 6/15
31/31 [==============================] - 371s 12s/step - loss: 0.4871 - accuracy: 0.7448 - val_loss: 0.5866 - val_accuracy: 0.6865
Epoch 7/15
31/31 [==============================] - 415s 13s/step - loss: 0.4367 - accuracy: 0.7707 - val_loss: 0.5938 - val_accuracy: 0.6854
Epoch 8/15
31/31 [==============================] - 478s 15s/step - loss: 0.3701 - accuracy: 0.8228 - val_loss: 0.6552 - val_accuracy: 0.6573
Epoch 9/15
31/31 [==============================] - 475s 15s/step - loss: 0.3330 - accuracy: 0.8487 - val_loss: 0.6219 - val_accuracy: 0.7198
Epoch 10/15
31/31 [==============================] - 478s 15s/step - loss: 0.2704 - accuracy: 0.8683 - val_loss: 0.7226 - val_accuracy: 0.7031
Epoch 11/15
31/31 [==============================] - 476s 15s/step - loss: 0.2363 - accuracy: 0.8998 - val_loss: 0.7711 - val_accuracy: 0.6885
Epoch 12/15
31/31 [==============================] - 443s 14s/step - loss: 0.1810 - accuracy: 0.9267 - val_loss: 0.8393 - val_accuracy: 0.6979
Epoch 13/15
31/31 [==============================] - 356s 11s/step - loss: 0.1441 - accuracy: 0.9473 - val_loss: 0.8402 - val_accuracy: 0.7167
Epoch 14/15
31/31 [==============================] - 344s 11s/step - loss: 0.1153 - accuracy: 0.9607 - val_loss: 0.9459 - val_accuracy: 0.7208
Epoch 15/15
31/31 [==============================] - 347s 11s/step - loss: 0.0952 - accuracy: 0.9675 - val_loss: 1.0133 - val_accuracy: 0.7177

模型训练集上的正确率：96.75%
模型验证集上的正确率：73.54%


